# 영화진흥위원회 OpenAPI 연동
- `JSON`을 불러와서 `excel`로 저장
- OPEN API같은 경우 하루에 연동할 수 있는 수가 제한되어 있다
    - 영화진흥위원회 같은 경우는 하루에 3000변만 연동 가능, 네이버는 2만번
- https://www.kofic.or.kr/
    - 해당 사이트에 가입해야지 Open API를 사용할 수 있다
    - 로그인 후 키 발급/관리 에서 키(API 연동 키) 발급 받기
    - OPEN API > REST 방식 선택
        - SOAP는 잘 사용 안 함
        - 기본 요청 URL의 주소가 .xml 또는 .json이라고 써있는데 .json이 경량의 데이터이므로 (i.e., 글자 수가 적어서 더 전송 속도가 빠름) 되도록이면 .json 추천 (.xml은 .html의 조상)
- `pandas`, `openpyxl`, `xlrd` 모듈의 설치 필요
    - pandas: 통계 분석할 때 사용하는 대표적인 라이브러리
    - openpyxl과 xlrd는 엑셀을 다루는 라이브러리

```shell
$ pip3 install --upgrade pandas
$ pip3 install --upgrade openpyxl
$ pip3 install --upgrade xlrd
```

## 1. 필요한 모듈 참조

In [1]:
# 필요한 모듈 참조
import requests
import json
import datetime as dt
from pandas import DataFrame

## 2. 최종 url 객체 생성 (`url 기본 형식 + 요청 인터페이스`)
- 요청 인터페이스 중 필수로 들어가야 하는 요청 변수들 (i.e., 파라미터들):
    - `key`: 발급받은 키 값 입력
    - `targetDt`: 조회하고자 하는 날짜를 yyyymmdd 형식으로 입력
### 1) `url 기본 형식` 객체 생성
- url 상 `get 파라미터`의 경우 `?` 다음에 넣고 여러개일 경우 중간에 `&`를 넣는다
- 문법 예시:
    - 'https://www.kobis ... .json`?`key={발급받은키}`&`targetDt={원하는날짜}'

In [2]:
# 온라인 상의 URL 기본 형식 (.json 확장자 사용)
urlFmt = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key={key}&targetDt={targetDt}"

### 2) `발급받은 API 연동 키 (key)` 객체 생성

In [3]:
# 발급받은 API 연동키
API_KEY = "6d2cf4aa96725383235c717f2e569f1e"

### 3) `날짜 (targetDt)` 객체 생성

In [4]:
# 하루 전 날짜 얻기
    # 집계 데이터 같은 경우 해당 날의 집계는 다음 날이 되어야 볼 수 있는 경우가 대부분이기에 하루 전 날짜 얻기
date = dt.datetime.now() - dt.timedelta(days = 1)
yesterday = date.strftime("%Y%m%d")
yesterday

'20231217'

### 4) 최종 url 객체 생성

In [5]:
# 최종 URL 얻기
url = urlFmt.format(key=API_KEY, targetDt=yesterday)
url

'http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=6d2cf4aa96725383235c717f2e569f1e&targetDt=20231217'

## 3. 접속 객체 생성 및 접속 정보 설정

In [6]:
# 접속 객체 생성
session = requests.Session()

# 접속 정보 설정
session.headers.update({
    "Referer": "",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
})

## 4. 원하는 웹페이지에 접속

In [7]:
# 특정 웹 페이지에 접속
r = session.get(url)

# 접속에 실패한 경우
if r.status_code != 200:
    msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
    raise Exception(msg)

## 5. json 형식을 딕셔너리로 변환

In [8]:
# 인코딩 형식 지정
r.encoding = "utf-8"

# 가져온 문자열을 딕셔너리로 변환
data = json.loads(r.text)
data

{'boxOfficeResult': {'boxofficeType': '일별 박스오피스',
  'showRange': '20231217~20231217',
  'dailyBoxOfficeList': [{'rnum': '1',
    'rank': '1',
    'rankInten': '0',
    'rankOldAndNew': 'OLD',
    'movieCd': '20212866',
    'movieNm': '서울의 봄',
    'openDt': '2023-11-22',
    'salesAmt': '4482576400',
    'salesShare': '72.0',
    'salesInten': '-545722420',
    'salesChange': '-10.9',
    'salesAcc': '86502037671',
    'audiCnt': '445492',
    'audiInten': '-51598',
    'audiChange': '-10.4',
    'audiAcc': '8940972',
    'scrnCnt': '2259',
    'showCnt': '8887'},
   {'rnum': '2',
    'rank': '2',
    'rankInten': '0',
    'rankOldAndNew': 'OLD',
    'movieCd': '20234673',
    'movieNm': '뽀로로 극장판 슈퍼스타 대모험',
    'openDt': '2023-12-13',
    'salesAmt': '449388441',
    'salesShare': '7.2',
    'salesInten': '189653',
    'salesChange': '0',
    'salesAcc': '1443402598',
    'audiCnt': '48800',
    'audiInten': '-273',
    'audiChange': '-0.6',
    'audiAcc': '159720',
    'scrnCnt': '850'

## 6. 원하는 부분만 추출
- 똑같은 딕셔너리를 원소로 갖는 리스트 추출

In [9]:
# 필요한 부분만 추출하기
dailyBoxOfficeList = data['boxOfficeResult']['dailyBoxOfficeList']
dailyBoxOfficeList

[{'rnum': '1',
  'rank': '1',
  'rankInten': '0',
  'rankOldAndNew': 'OLD',
  'movieCd': '20212866',
  'movieNm': '서울의 봄',
  'openDt': '2023-11-22',
  'salesAmt': '4482576400',
  'salesShare': '72.0',
  'salesInten': '-545722420',
  'salesChange': '-10.9',
  'salesAcc': '86502037671',
  'audiCnt': '445492',
  'audiInten': '-51598',
  'audiChange': '-10.4',
  'audiAcc': '8940972',
  'scrnCnt': '2259',
  'showCnt': '8887'},
 {'rnum': '2',
  'rank': '2',
  'rankInten': '0',
  'rankOldAndNew': 'OLD',
  'movieCd': '20234673',
  'movieNm': '뽀로로 극장판 슈퍼스타 대모험',
  'openDt': '2023-12-13',
  'salesAmt': '449388441',
  'salesShare': '7.2',
  'salesInten': '189653',
  'salesChange': '0',
  'salesAcc': '1443402598',
  'audiCnt': '48800',
  'audiInten': '-273',
  'audiChange': '-0.6',
  'audiAcc': '159720',
  'scrnCnt': '850',
  'showCnt': '1594'},
 {'rnum': '3',
  'rank': '3',
  'rankInten': '0',
  'rankOldAndNew': 'OLD',
  'movieCd': '20190324',
  'movieNm': '3일의 휴가',
  'openDt': '2023-12-06',
  's

## 7. DataFrame으로 결과표 생성

In [10]:
# 결과 표 생성하기
df = DataFrame(dailyBoxOfficeList)
df

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt
0,1,1,0,OLD,20212866,서울의 봄,2023-11-22,4482576400,72.0,-545722420,-10.9,86502037671,445492,-51598,-10.4,8940972,2259,8887
1,2,2,0,OLD,20234673,뽀로로 극장판 슈퍼스타 대모험,2023-12-13,449388441,7.2,189653,0,1443402598,48800,-273,-0.6,159720,850,1594
2,3,3,0,OLD,20190324,3일의 휴가,2023-12-06,364528712,5.9,-28905500,-7.3,4026993896,36426,-2942,-7.5,425865,801,1887
3,4,4,0,OLD,20234114,괴물,2023-11-29,165084507,2.7,-13631409,-7.6,2778565838,15805,-1363,-7.9,288601,401,635
4,5,5,1,OLD,20235980,말하고 싶은 비밀,2023-12-13,111810808,1.8,6098110,5.8,441180290,11100,572,5.4,45696,392,586
5,6,6,-1,OLD,20235698,쏘우 X,2023-12-13,102059505,1.6,-23630967,-18.8,524330177,9414,-2324,-19.8,50318,483,776
6,7,7,0,OLD,20235098,나폴레옹,2023-12-06,90378181,1.5,-8852295,-8.9,2089597868,9161,-789,-7.9,208102,482,632
7,8,8,2,OLD,20235030,프레디의 피자가게,2023-11-15,67175743,1.1,-4786608,-6.7,6934650101,6529,-401,-5.8,709645,217,280
8,9,9,0,OLD,20202247,싱글 인 서울,2023-11-29,61128020,1.0,-10879389,-15.1,3569191206,6108,-1042,-14.6,381205,365,472
9,10,10,-2,OLD,20203702,노량: 죽음의 바다,2023-12-20,44706400,0.7,-42615210,-48.8,274591010,5087,-4112,-44.7,27503,15,31


## 8. 데이터 저장하기

In [11]:
# 엑셀로 저장
df.to_excel("박스오피스_순위_%s.xlsx" % yesterday)